In [1]:
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langchain.agents import create_agent, create_tool
from langchain.messages import HumanMessage, AIMessage, ToolMessage
from langchain.tools import tool
from langchain_community.document_loaders.csv_loader import  CSVLoader

2026-02-27 11:43:20.471991: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-27 11:43:20.604445: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-02-27 11:43:23.719676: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/home/azizbek/Python/Projects/Laws-Downloader/venv/lib/python3.12/site-packages/

# Test with class CSVLoader

In [2]:
loader = CSVLoader(file_path="data/diabetes.csv")

In [3]:
data = loader.load()

In [4]:
print(type(data))

<class 'list'>


In [5]:
len(data)

768

In [6]:
load_dotenv()

True

In [48]:
import pandas as pd

df = pd.read_csv("data/diabetes.csv")

In [27]:
@tool
def set_index(column_as_index: str) -> str:

    """
    Bu metod dataframega indeks o'rnatadi
    """
    global df
    df = df.set_index(column_as_index)
    return f"{column_as_index} indeks sifatida o'rnatildi"


In [28]:
model = init_chat_model(model="gemini-2.5-flash", model_provider="google_genai")
agent = create_agent(
    model=model,
    tools=[set_index],
    system_prompt="Foydalanuvchi so'roviga mos vazifani bajar.",
)

In [29]:
response = agent.invoke({
    "messages": [
        {
            "role": "user",
            "content": "Pregnancies ustunni indeks sifatida o'rnat."
        }
    ]})

In [30]:
response["messages"][-1].content

"Pregnancies ustuni indeks sifatida o'rnatildi."

In [31]:
df

,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
Pregnancies,,,,,,,,
6,148,72,35,0,33.6,0.627,50,1
1,85,66,29,0,26.6,0.351,31,0
8,183,64,0,0,23.3,0.672,32,1
1,89,66,23,94,28.1,0.167,21,0
0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...
10,101,76,48,180,32.9,0.171,63,0
2,122,70,27,0,36.8,0.340,27,0
5,121,72,23,112,26.2,0.245,30,0


In [36]:
@tool
def change_column_name(last_name: str, new_name: str) -> str:

    '''Bir ustun nomnini o'zgartirish'''
    global df

    df = df.rename(columns={last_name: new_name})
    return f"{last_name} ustun nomi {new_name} o'zgartirildi."

In [37]:
model = init_chat_model(model="gemini-2.5-flash", model_provider="google_genai")
agent = create_agent(
    model=model,
    tools=[set_index, change_column_name],
    system_prompt="Foydalanuvchi so'roviga mos vazifani bajar.",
)

In [38]:

response = agent.invoke({
    "messages": [
        {
            "role": "user",
            "content": "'Insulin' ustun nomini 'Kasallar' ga o'zgartir."
        }
    ]})

In [42]:
response["messages"][-1].content

'"Insulin" ustun nomi "Kasallar" ga o\'zgartirildi.'

In [43]:
df

,Glucose,BloodPressure,SkinThickness,Kasallar,BMI,DiabetesPedigreeFunction,Age,Outcome
Pregnancies,,,,,,,,
6,148,72,35,0,33.6,0.627,50,1
1,85,66,29,0,26.6,0.351,31,0
8,183,64,0,0,23.3,0.672,32,1
1,89,66,23,94,28.1,0.167,21,0
0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...
10,101,76,48,180,32.9,0.171,63,0
2,122,70,27,0,36.8,0.340,27,0
5,121,72,23,112,26.2,0.245,30,0


In [40]:
@tool
def reset_index():

    """Bu metod dataframedagi indeksni olib tashlaydi"""

    global df
    df = df.reset_index(drop=False)

    return f"Dataframedan indeks olib tashkandi"

In [44]:
model = init_chat_model(model="gemini-2.5-flash", model_provider="google_genai")
agent = create_agent(
    model=model,
    tools=[set_index, reset_index, change_column_name],
    system_prompt="Foydalanuvchi so'roviga mos vazifani bajar.",
)

In [45]:
response = agent.invoke({
    "messages": [HumanMessage(content="Dataframedan indeksni olib tashla.")],
})

In [46]:
response["messages"][-1].content

'Dataframedan indeks olib tashkandi'

In [49]:
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [50]:
response = agent.invoke({
    "messages": [HumanMessage(content="Dataframega 'BMI' ustunini indeks sifatida o'rnat va 'Insulin' ustun nomini 'Kasallar'ga o'zgartir." )],
})

In [53]:
response["messages"][-1].content

"Dataframega 'BMI' ustuni indeks sifatida o'rnatildi va 'Insulin' ustun nomi 'Kasallar'ga o'zgartirildi."

In [54]:
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Kasallar,DiabetesPedigreeFunction,Age,Outcome
BMI,,,,,,,,
33.6,6,148,72,35,0,0.627,50,1
26.6,1,85,66,29,0,0.351,31,0
23.3,8,183,64,0,0,0.672,32,1
28.1,1,89,66,23,94,0.167,21,0
43.1,0,137,40,35,168,2.288,33,1
...,...,...,...,...,...,...,...,...
32.9,10,101,76,48,180,0.171,63,0
36.8,2,122,70,27,0,0.340,27,0
26.2,5,121,72,23,112,0.245,30,0
